# Sentinel Playground Temporal

A graphical interface to a complete and daily updated Sentinel-2 archive

Link: https://apps.sentinel-hub.com/sentinel-playground-temporal


### Activity:
- Go to Sentinel Playground
- Select a location (Philadelphia?)
- Display it in real color
- Display different color bands   

## Custom scripting, simple usage:

Playground provides a Javascript based interface for visualization.
 
Custom scripts: Javascript code used to control the visualization and processing of satellite images with
Sentinel Hub



### Activity:
- Select a location (Mount Etna)
- Display it using the following script:

return [
    B04 * 2.5 + Math.max(0, B12 - 0.1),
    B03 * 2.5 + Math.max(0, B11 - 0.1),
    B02 * 2.5
];

## Map algebra

<img src="Images/mapAlgebra.png">


## Custom scripting, basic functions:

<img src="Images/img1.png">


### Activity: Visualize Etna Eruption
- Select a location (Etna Parco Regio)
- Select a date (2017-03-16)
- Display it using the following script:



#### Simple script (True color)

In [1]:
return [ B04 * 2.5, B03 * 2.5, B02 * 2.5 ];

SyntaxError: 'return' outside function (<ipython-input-1-366388989b1c>, line 1)

#### Simple script (False color, Vegetation in Red channel)

In [ ]:
return [ B08 * 2.5, B04 * 2.5, B03 * 2.5 ];

#### Simple script (Highlight lava)

In [ ]:
return [ B12 * 2.5, B11 * 2.5, B04 * 2.5 ];

####  Custom script

In [2]:
function setup(ds) {
    setInputComponents([ds.B02, ds.B03, ds.B04, ds.B11, ds.B12]);
    setOutputComponentCount(3);
}

let viz = new HighlightCompressVisualizerSingle(-1.0, 1.0);

function evaluatePixel(samples) {
    let val1 = samples[0].B04 * 2.5 + Math.max(0, samples[0].B12 - 0.1)
    let val2 = samples[0].B03 * 2.5 + Math.max(0, samples[0].B11 - 0.1)
    let val3 = samples[0].B02 * 2.5
    return [val1, val2, val3]; 
}

SyntaxError: invalid syntax (<ipython-input-2-0006c9d68d4a>, line 1)

<img src="Images/shub1_Etna.png">

<img src="Images/shub2_Etna.png">

## Creating an Index

### NDVI =  (𝑁𝐼𝑅−𝑅𝐸𝐷) / (𝑁𝐼𝑅+𝑅𝐸𝐷)

let NDVI = 2.5 * ((B08 - B04) / (B08 + B04));
return [ NDVI ]
or
return [0, NDVI, 0]


## Index function

In [ ]:
let NDVI = index ( B08 ,  B04 ); //   index(X,Y) -> (X-Y)/(X+Y) 


### List of indexes

<img src="Images/listIndex.png">


## Custom color scales

### Activity: Discrete color scale for NDVI
    

In [ ]:
var NDVI = index (B08, B04); 
if (NDVI < 0.2) {
 return [0.3, 0.2, 0.7]
}
if (NDVI < 0.5) {
 return [0.2, 0.6, 0.3]
}
if (NDVI < 0.7) {
 return [0.5, 0.8, 0.2]
}
else {
 return [1, 0.4, 0]
}

### Activity: Continuous color scale for NDVI
    

In [ ]:
var NDVI = index (B08, B04);
return colorBlend
(NDVI, // Pixel value
 [0, 0.5, 1], // Define the borders
 [ [0, 0, 1], // Define the RGB colors for each border
 [0, 1, 0],
 [1, 0, 0],
 ]);

<img src="Images/phl2.png">

## Temporal visualization

### Activity: Sydney, wildfires

#### Important: Activate temporal on the "effects pane"!

#### Setup function (set input/output)

In [ ]:
function setup (dss) {
  // get all bands for display and analysis
  setInputComponents([dss.B02,dss.B03,dss.B04,dss.B05,dss.B08,dss.B12]);
  // return as RGB
  setOutputComponentCount(3);
}

#### Stretch function (rescale values)

In [ ]:
function stretch(val, min, max) {return (val - min) / (max - min);}

#### filterScenes function (select multiple timepoints)

In [ ]:
function filterScenes (scenes, inputMetadata) {  
return scenes.filter(function (scene) {
// set dates for pre-and-post fire analysis
var allowedDates = ["2019-11-01","2019-12-26"]; // Australia fires  

// format scene date timestamp to match allowed dates 
var sceneDateStr = dateformat(scene.date);
if (allowedDates.indexOf(sceneDateStr)!= -1) return true;
else return false;
  });
}

function dateformat(d){  
  var dd = d.getDate();
  var mm = d.getMonth()+1;
  var yyyy = d.getFullYear();
  if(dd<10){dd='0'+dd}
  if(mm<10){mm='0'+mm}
  var isodate = yyyy+'-'+mm+'-'+dd;
  return isodate;
}

#### calcNBR function

In [ ]:
// Normalized Burn Ration calculation
function calcNBR(sample) {
  var denom = sample.B08+sample.B12;
  var nbrval = ((denom!=0) ? (sample.B08-sample.B12) / denom : 0.0);
  return nbrval;
}

#### evaluatePixel function

In [ ]:
function evaluatePixel(samples,scenes) {  
  var nbrpre = 0;
  var nbrpost = 0;  
  
  // get pre-fire image
  nbrpre = calcNBR(samples[1]);
  // get post-fire image
  nbrpost = calcNBR(samples[0]);  
  // get difference 
  var dnbr = nbrpre - nbrpost;
  // set output display layers
  var stretchMin = 0.05;
  var stretchMax = 1.00;
  var NaturalColors = [stretch(2.8 * samples[0].B04 + 0.1 * samples[0].B05, stretchMin, stretchMax), stretch(2.8 * samples[0].B03 + 0.15 * samples[0].B08, stretchMin, stretchMax), stretch(2.8 * samples[0].B02, stretchMin, stretchMax)];  
  var burnModerate = [stretch(2.8 * samples[0].B04 + 0.1 * samples[0].B05, stretchMin, stretchMax)+0.5, stretch(2.8 * samples[0].B03 + 0.15 * samples[0].B08, stretchMin, stretchMax)+0.5, stretch(2.8 * samples[0].B02, stretchMin, stretchMax)];  
  var burnSevere = [stretch(2.8 * samples[0].B04 + 0.1 * samples[0].B05, stretchMin, stretchMax)+0.5, stretch(2.8 * samples[0].B03 + 0.15 * samples[0].B08, stretchMin, stretchMax), stretch(2.8 * samples[0].B02, stretchMin, stretchMax)];
  return (dnbr < 0.27 ?
  NaturalColors : (dnbr < 0.66 ?
  burnModerate : burnSevere)
  ); 
}

#### Complete script:

In [ ]:
function setup (dss) {
  // get all bands for display and analysis
  setInputComponents([dss.B02,dss.B03,dss.B04,dss.B05,dss.B08,dss.B12]);
  // return as RGB
  setOutputComponentCount(3);
}

function stretch(val, min, max) {return (val - min) / (max - min);}

function filterScenes (scenes, inputMetadata) {  
return scenes.filter(function (scene) {
// set dates for pre-and-post fire analysis
var allowedDates = ["2019-11-01","2019-12-26"]; // Australia fires
// format scene date timestamp to match allowed dates 
var sceneDateStr = dateformat(scene.date);
if (allowedDates.indexOf(sceneDateStr)!= -1) return true;
else return false;
  });
}

// Normalized Burn Ration calculation
function calcNBR(sample) {
  var denom = sample.B08+sample.B12;
  var nbrval = ((denom!=0) ? (sample.B08-sample.B12) / denom : 0.0);
  return nbrval;
}

function dateformat(d){  
  var dd = d.getDate();
  var mm = d.getMonth()+1;
  var yyyy = d.getFullYear();
  if(dd<10){dd='0'+dd}
  if(mm<10){mm='0'+mm}
  var isodate = yyyy+'-'+mm+'-'+dd;
  return isodate;
}

function evaluatePixel(samples,scenes) {  
  var nbrpre = 0;
  var nbrpost = 0;  
  
  // get pre-fire image
  nbrpre = calcNBR(samples[1]);
  // get post-fire image
  nbrpost = calcNBR(samples[0]);  
  // get difference 
  var dnbr = nbrpre - nbrpost;
  // set output display layers
  var stretchMin = 0.05;
  var stretchMax = 1.00;
  var NaturalColors = [stretch(2.8 * samples[0].B04 + 0.1 * samples[0].B05, stretchMin, stretchMax), stretch(2.8 * samples[0].B03 + 0.15 * samples[0].B08, stretchMin, stretchMax), stretch(2.8 * samples[0].B02, stretchMin, stretchMax)];  
  var burnModerate = [stretch(2.8 * samples[0].B04 + 0.1 * samples[0].B05, stretchMin, stretchMax)+0.5, stretch(2.8 * samples[0].B03 + 0.15 * samples[0].B08, stretchMin, stretchMax)+0.5, stretch(2.8 * samples[0].B02, stretchMin, stretchMax)];  
  //var burnLow = [1, 204/255, 0]
  var burnModerate = [1, 204/255, 0]
  var burnHigh = [255/255, 121/255, 77/255]
  var burnVeryHigh = [stretch(2.8 * samples[0].B04 + 0.1 * samples[0].B05, stretchMin, stretchMax)+0.5, stretch(2.8 * samples[0].B03 + 0.15 * samples[0].B08, stretchMin, stretchMax), stretch(2.8 * samples[0].B02, stretchMin, stretchMax)];
  //Classification based on http://effis.jrc.ec.europa.eu/about-effis/technical-background/rapid-damage-assessment/
  return (dnbr < 0.256 ?
  NaturalColors : (dnbr < 0.42 ?
  burnModerate : (dnbr < 0.66 ?
  burnHigh : burnVeryHigh)));
}


<img src="Images/shub3_Sydney.png">

## Sentinel Hub Custom Script Contest

Many examples!

Unfortunately, deadline missed this year!

https://sentinel-hub.com/contest

## Homework 1

[Storytelling using satellite imaging](https://github.com/MUSA-650-Spring-2020/Week1/blob/master/HW1.ipynb)

 